In [1]:
!pip install kaggle

from google.colab import files
files.upload() 

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d tongpython/cat-and-dog

!unzip -q cat-and-dog.zip -d cat_and_dog/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json
 99% 216M/218M [00:01<00:00, 151MB/s]
100% 218M/218M [00:01<00:00, 117MB/s]


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader 
from torch.utils.data import random_split
from torchvision import datasets 
from torchvision import transforms 
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

In [3]:
dir = './cat_and_dog/'
transform = transforms.Compose([transforms.Resize([224, 224]), transforms.ToTensor()])
# cats와 dogs의 위치 폴더가 다르므로 다른 클래스로 인식함
trainset = datasets.ImageFolder(dir+'training_set/training_set', transform = transform)
testset = datasets.ImageFolder(dir+'test_set/test_set', transform = transform)
len(trainset), len(testset)

(8005, 2023)

In [5]:
from google.colab import drive
drive.mount('/content/drive')
import os
%cd '/content/drive/MyDrive/Colab Notebooks'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [6]:
from NN_model import print_models, AlexNet
from NN_train import train
print_models()

ori_DNN, ori_CNN, ori2_CNN, CIFAR10_CNN, AlexNet, VGG16, VGG16_2, ResNet


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [8]:
train_idxs, valid_idxs, _, _ = train_test_split(
                            range(len(trainset)), trainset.targets, test_size= 0.2, random_state=42,
                            stratify=trainset.targets) # target의 비율이 train과 valid에 그대로 반영되게
len(train_idxs), len(valid_idxs) # 80%, 20%

(6404, 1601)

In [ ]:
train_set = Subset(trainset, train_idxs)
valid_set = Subset(trainset, valid_idxs)
train_set[0][0].size(), train_set[0][1]

(torch.Size([3, 224, 224]), 0)

In [ ]:
batch_s = 16 # 100 -> 16
trainloader = DataLoader(train_set, batch_size=batch_s, shuffle=True) 
validloader = DataLoader(valid_set, batch_size=batch_s, shuffle=False)
testloader = DataLoader(testset, batch_size=batch_s, shuffle=False)
len(trainloader), len(validloader), len(testloader)

(401, 101, 127)

In [ ]:
train_iter = iter(trainloader)
imgs, labels = next(train_iter)
imgs.size(), labels.size()

(torch.Size([16, 3, 224, 224]), torch.Size([16]))

In [ ]:
lr = 0.001; patience = 7; epochs = 55
model = AlexNet()
model = model.to(device)
loss_fn = nn.CrossEntropyLoss() # loss function
optimizer = optim.Adam(model.parameters(), lr=lr) # optimizer
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4, verbose=True)

myNN = train(model, loss_fn, optimizer, trainloader, validloader, testloader, 
             scheduler, device, epochs, patience, batch_s)
myNN.train_loop()

KeyboardInterrupt: 

In [ ]:
myNN.result_plot()

In [ ]:
myNN.check(unsq=True, rand_idx=10)

In [ ]:
myNN.evaluate(valid=False)

In [ ]:
torch.save(model.state_dict(), 'last_checkpoint.pth')
myNN.load_model(dirct='last_checkpoint.pth', model)

In [ ]:
myNN.load_model(dirct='best_checkpoint.pth', model)